In [ ]:
import pandas as pd
import numpy as np

def analyze_third_candle_movement(file_path):
    """
    Analyze tick-based candle data to find patterns and calculate average movement
    in the third candles for both green and red patterns.
    
    Args:
        file_path: Path to the CSV file with tick-based candle data
    
    Returns:
        Dictionary with pattern statistics and movement data
    """
    # Load the candle data
    df = pd.read_csv(file_path)
    
    # Ensure the DataFrame is sorted by time
    if 'datetime' in df.columns:
        df = df.sort_values('datetime')
    else:
        df = df.sort_values('timestamp')
    
    # Calculate if candles are green (close > open) or red (close < open)
    df['is_green'] = df['close'] > df['open']
    df['is_red'] = df['close'] < df['open']
    
    # Calculate absolute and percentage movements
    df['abs_movement'] = abs(df['close'] - df['open'])
    df['pct_movement'] = df['abs_movement'] / df['open'] * 100
    df['high_low_range'] = df['high'] - df['low']
    
    # Calculate EMAs
    df['ema9'] = df['close'].ewm(span=9, adjust=False).mean()
    df['ema15'] = df['close'].ewm(span=15, adjust=False).mean()
    
    # Create EMA conditions
    df['green_ema_condition'] = df['ema9'] > df['ema15']  # For green candle patterns
    df['red_ema_condition'] = df['ema9'] < df['ema15']    # For red candle patterns
    
    # Lists to store third candle movements
    green_pattern_third_candle_movements = []
    green_pattern_third_candle_ranges = []
    red_pattern_third_candle_movements = []
    red_pattern_third_candle_ranges = []
    
    # Initialize counters and result data for green patterns
    total_green_third_candles = 0
    green_third_candles = 0
    
    # Initialize counters and result data for red patterns
    total_red_third_candles = 0
    red_third_candles = 0
    
    # State variables for pattern tracking
    green_pattern_state = 0  # 0: looking for 1st green, 1: found 1st green, 2: found 2nd green, 3: analyzing 3rd
    red_pattern_state = 0    # 0: looking for 1st red, 1: found 1st red, 2: found 2nd red, 3: analyzing 3rd
    
    # Loop through the dataframe to find patterns
    for i in range(len(df)):
        current_candle_green = df['is_green'].iloc[i]
        current_candle_red = df['is_red'].iloc[i]
        green_ema_condition_met = df['green_ema_condition'].iloc[i]
        red_ema_condition_met = df['red_ema_condition'].iloc[i]
        
        # Process GREEN pattern states
        if green_pattern_state == 0:
            # Looking for first green candle
            if current_candle_green:
                green_pattern_state = 1
        elif green_pattern_state == 1:
            # Looking for second green candle
            if current_candle_green:
                green_pattern_state = 2
            else:
                green_pattern_state = 0  # Reset if not green
        elif green_pattern_state == 2:
            # Checking third candle, but only if EMA condition is met
            if green_ema_condition_met:
                total_green_third_candles += 1
                
                # Record movement data for this third candle
                current_abs_movement = df['abs_movement'].iloc[i]
                current_high_low_range = df['high_low_range'].iloc[i]
                green_pattern_third_candle_movements.append(current_abs_movement)
                green_pattern_third_candle_ranges.append(current_high_low_range)
                
                if current_candle_green:
                    green_third_candles += 1
                green_pattern_state = 3  # Move to next state regardless of third candle color
            else:
                green_pattern_state = 0  # Reset if EMA condition not met
        elif green_pattern_state == 3:
            # If fourth candle is green, use it as first candle of new pattern
            if current_candle_green:
                green_pattern_state = 1
            else:
                green_pattern_state = 0
        
        # Process RED pattern states
        if red_pattern_state == 0:
            # Looking for first red candle
            if current_candle_red:
                red_pattern_state = 1
        elif red_pattern_state == 1:
            # Looking for second red candle
            if current_candle_red:
                red_pattern_state = 2
            else:
                red_pattern_state = 0  # Reset if not red
        elif red_pattern_state == 2:
            # Checking third candle, but only if EMA condition is met
            if red_ema_condition_met:
                total_red_third_candles += 1
                
                # Record movement data for this third candle
                current_abs_movement = df['abs_movement'].iloc[i]
                current_high_low_range = df['high_low_range'].iloc[i]
                red_pattern_third_candle_movements.append(current_abs_movement)
                red_pattern_third_candle_ranges.append(current_high_low_range)
                
                if current_candle_red:
                    red_third_candles += 1
                red_pattern_state = 3  # Move to next state regardless of third candle color
            else:
                red_pattern_state = 0  # Reset if EMA condition not met
        elif red_pattern_state == 3:
            # If fourth candle is red, use it as first candle of new pattern
            if current_candle_red:
                red_pattern_state = 1
            else:
                red_pattern_state = 0
    
    # Calculate movement statistics
    green_avg_movement = np.mean(green_pattern_third_candle_movements) if green_pattern_third_candle_movements else 0
    green_median_movement = np.median(green_pattern_third_candle_movements) if green_pattern_third_candle_movements else 0
    green_max_movement = np.max(green_pattern_third_candle_movements) if green_pattern_third_candle_movements else 0
    green_min_movement = np.min(green_pattern_third_candle_movements) if green_pattern_third_candle_movements else 0
    
    green_avg_range = np.mean(green_pattern_third_candle_ranges) if green_pattern_third_candle_ranges else 0
    green_median_range = np.median(green_pattern_third_candle_ranges) if green_pattern_third_candle_ranges else 0
    
    red_avg_movement = np.mean(red_pattern_third_candle_movements) if red_pattern_third_candle_movements else 0
    red_median_movement = np.median(red_pattern_third_candle_movements) if red_pattern_third_candle_movements else 0
    red_max_movement = np.max(red_pattern_third_candle_movements) if red_pattern_third_candle_movements else 0
    red_min_movement = np.min(red_pattern_third_candle_movements) if red_pattern_third_candle_movements else 0
    
    red_avg_range = np.mean(red_pattern_third_candle_ranges) if red_pattern_third_candle_ranges else 0
    red_median_range = np.median(red_pattern_third_candle_ranges) if red_pattern_third_candle_ranges else 0
    
    # Calculate average price for context
    avg_price = df['close'].mean()
    
    # Calculate movement as percentage of average price
    green_avg_movement_pct = (green_avg_movement / avg_price) * 100
    red_avg_movement_pct = (red_avg_movement / avg_price) * 100
    
    # Calculate suggested take profit and stop loss values
    # A common approach is using the average movement for take profit
    # and a fraction of it for stop loss (e.g., 50-75% of average movement)
    green_suggested_tp = green_avg_movement
    green_suggested_sl = green_avg_movement * 0.6  # 60% of average movement
    
    red_suggested_tp = red_avg_movement
    red_suggested_sl = red_avg_movement * 0.6  # 60% of average movement
    
    # Calculate success rate percentages
    green_success_rate = (green_third_candles / total_green_third_candles * 100) if total_green_third_candles > 0 else 0
    red_success_rate = (red_third_candles / total_red_third_candles * 100) if total_red_third_candles > 0 else 0
    
    # Prepare results
    results = {
        # Green pattern results
        'green_total_patterns_found': total_green_third_candles,
        'green_third_candles': green_third_candles,
        'green_success_rate_percentage': green_success_rate,
        'green_avg_movement': green_avg_movement,
        'green_median_movement': green_median_movement,
        'green_max_movement': green_max_movement,
        'green_min_movement': green_min_movement,
        'green_avg_range': green_avg_range,
        'green_median_range': green_median_range,
        'green_avg_movement_pct': green_avg_movement_pct,
        'green_suggested_tp': green_suggested_tp,
        'green_suggested_sl': green_suggested_sl,
        
        # Red pattern results
        'red_total_patterns_found': total_red_third_candles,
        'red_third_candles': red_third_candles,
        'red_success_rate_percentage': red_success_rate,
        'red_avg_movement': red_avg_movement,
        'red_median_movement': red_median_movement,
        'red_max_movement': red_max_movement,
        'red_min_movement': red_min_movement,
        'red_avg_range': red_avg_range,
        'red_median_range': red_median_range,
        'red_avg_movement_pct': red_avg_movement_pct,
        'red_suggested_tp': red_suggested_tp,
        'red_suggested_sl': red_suggested_sl
    }
    
    return results

def main():
    # File path to your saved tick-based candle data
    file_path = '/Users/sushanth/Desktop/Nano_scalping/Crypto/btcusdc_30tick_candles.csv'  # Update this to match your saved file
    
    try:
        # Analyze the pattern and movements
        results = analyze_third_candle_movement(file_path)
        
        # Print results for green candle patterns
        print("\nGREEN CANDLE PATTERN RESULTS:")
        print(f"Total green patterns found: {results['green_total_patterns_found']}")
        print(f"Green third candles: {results['green_third_candles']}")
        print(f"Green pattern success rate: {results['green_success_rate_percentage']:.2f}%")
        
        # Print movement statistics for green patterns
        print("\nGREEN THIRD CANDLE MOVEMENT STATISTICS:")
        print(f"Average movement (open to close): {results['green_avg_movement']:.8f} USDC")
        print(f"Median movement: {results['green_median_movement']:.8f} USDC")
        print(f"Max movement: {results['green_max_movement']:.8f} USDC")
        print(f"Min movement: {results['green_min_movement']:.8f} USDC")
        print(f"Average high-low range: {results['green_avg_range']:.8f} USDC")
        print(f"Movement as % of avg price: {results['green_avg_movement_pct']:.4f}%")
        
        # Print suggestions for green patterns
        print("\nGREEN PATTERN TRADE SUGGESTIONS:")
        print(f"Suggested take profit: {results['green_suggested_tp']:.8f} USDC")
        print(f"Suggested stop loss: {results['green_suggested_sl']:.8f} USDC")
        
        # Print results for red candle patterns
        print("\nRED CANDLE PATTERN RESULTS:")
        print(f"Total red patterns found: {results['red_total_patterns_found']}")
        print(f"Red third candles: {results['red_third_candles']}")
        print(f"Red pattern success rate: {results['red_success_rate_percentage']:.2f}%")
        
        # Print movement statistics for red patterns
        print("\nRED THIRD CANDLE MOVEMENT STATISTICS:")
        print(f"Average movement (open to close): {results['red_avg_movement']:.8f} USDC")
        print(f"Median movement: {results['red_median_movement']:.8f} USDC")
        print(f"Max movement: {results['red_max_movement']:.8f} USDC")
        print(f"Min movement: {results['red_min_movement']:.8f} USDC")
        print(f"Average high-low range: {results['red_avg_range']:.8f} USDC")
        print(f"Movement as % of avg price: {results['red_avg_movement_pct']:.4f}%")
        
        # Print suggestions for red patterns
        print("\nRED PATTERN TRADE SUGGESTIONS:")
        print(f"Suggested take profit: {results['red_suggested_tp']:.8f} USDC")
        print(f"Suggested stop loss: {results['red_suggested_sl']:.8f} USDC")
        
    except Exception as e:
        print(f"Error analyzing candle data: {str(e)}")

if __name__ == "__main__":
    main()